In [1]:
import os
import glob
import pandas as pd
import numpy as np
from copy import deepcopy

In [4]:
run_directory = r"C:\Users\pechen\Documents\XTMF\Projects\407 Model Update"

run_folder = r"407 Model - Warm Start- MCSpatial Run 43 - 20210610"
run_path = os.path.join(run_directory,run_folder)

input_pth = r"C:\Users\pechen\Documents\407 Model Update"
zone_map = pd.read_csv(os.path.join(input_pth,"407_ZoneMapping.csv"))
zone_map_min = zone_map

# Mode Share Start HERE

In [5]:
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

input_pth = r"C:\Users\pechen\Documents\407 Model Update"
zone_map = pd.read_csv(os.path.join(input_pth,"407_ZoneMapping.csv"))
zone_map_min = zone_map

persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
trips_gta = trips_df[(trips_df.o_zone < 6000) & (trips_df.d_zone < 6000)]

trips_gta = trips_gta.merge(persons_df,on=["household_id","person_id"])
trips_gta = trips_gta[trips_gta.age >=11]

trips_gta_time = tripmode_df.merge(trips_gta,how = "right", on = ['household_id', 'person_id', 'trip_id'])

trips_gta_time["weight"] = trips_gta_time["weight"]/10
trips_gta_time["weight_z"] = trips_gta_time["weight"]*trips_gta_time["weight_x"]

ms_daily_GTA = pd.pivot_table(trips_gta_time,columns = "mode", values = "weight_z", aggfunc = sum)

In [6]:
ms_daily_GTA.to_csv(os.path.join(input_pth,"GTA_Wide_TripsbyMode_MCSpatial_Run41.csv")) #Daily MS

In [7]:
trips_407 = trips_df
trips_407  = trips_407.merge(persons_df,on=["household_id","person_id"])
trips_407 = trips_407[trips_407.age >=11]
trips_407 = trips_407.merge(zone_map_min,left_on = "o_zone", right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_o"})
trips_407 = trips_407.merge(zone_map_min,left_on = "d_zone", right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_d"})

trips_407_time = tripmode_df.merge(trips_407,how = "right", on = ['household_id', 'person_id', 'trip_id'])
trips_407_time["weight"] = trips_407_time["weight"]/10
trips_407_time["weight_z"] = trips_407_time["weight"]*trips_407_time["weight_x"]
trips_407_am = trips_407_time[(trips_407_time.o_depart >= 360) & (trips_407_time.o_depart < 540)]
trips_407_pm = trips_407_time[(trips_407_time.o_depart >= 900) & (trips_407_time.o_depart < 1140)]

In [8]:
am_modeshare = pd.pivot_table(trips_407_am,index = ["region_o","region_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)
pm_modeshare = pd.pivot_table(trips_407_pm,index = ["region_o","region_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)

In [9]:
am_modeshare.to_csv(os.path.join(input_pth,"am_modeshare_MCSpatial_Run41.csv")) #Time Period MS
pm_modeshare.to_csv(os.path.join(input_pth,"pm_modeshare_MCSpatial_Run41.csv"))

# XML Edits for Mode Share Constants here

In [136]:
import xml.etree.ElementTree as ET
model_syst_pth = os.path.join(input_pth,"Python Model Systems")
constant_df = pd.read_csv(os.path.join(model_syst_pth,"407_msconstants_spatialrun23.csv"))
xml_fle = os.path.join(model_syst_pth,"407 Base Calibration - Warm Start Low Iter - MS Spatial Base 12 - 20210517.xml")

In [137]:
constant_df["OD"] = constant_df["O_PD"].map(str) + ":" + constant_df["D_PD"].map(str)
tree = ET.parse(xml_fle)
root = tree.getroot()
model = root[1]
otherModes = model[6]

for index, row in constant_df.iterrows():
    mode = row.Mode
    tp = row.TP
    od = row.OD
    const = row.Constant
    match = False
    for child in otherModes:
        curr_mode = child.attrib["Name"]
        if curr_mode == mode:
            mode_tree = child
            match = True
    if not match:
        print("No match found for {}".format(mode))
    else:
        match = False
        for child in mode_tree:
            if "Name" in child.attrib.keys():
                if child.attrib["Name"] == 'Time Period Constants':
                    tp_tree = child
                    match = True
        if not match:
            print("No time period constants for {}".format(mode))
        else:
            match = False
            for child in tp_tree:
                if "Name" in child.attrib.keys():
                    if child.attrib["Name"] == tp:
                        this_tp_tree = child[1]
                        match = True
            if not match:
                print("Time period {0} does not exist for mode {1}".format(tp,mode))
            else:
                match = False
                for child in this_tp_tree:
                    if "Name" in child.attrib.keys():
                        if child.attrib["Name"] == od:
                            match = True
                            const_tree = child
                if not match:
                    print("OD {0} not present for tp {1} mode {2}".format(od,tp,mode))
                else:
                    match = False
                    params = const_tree[0]
                    for child in params:
                        if child.attrib["Name"] == "Constant":
                            param = child
                            match = True
                    if not match:
                        print("Constant attribute not present for od {0} tp {1} mode {2}".format(od,tp,mode))
                    else:
                        param.attrib["Value"] = str(const)

In [138]:
tree.write(os.path.join(model_syst_pth,"407 Base Calibration - Warm Start Low Iter - MS Spatial Run 23 - 20210517.xml"))

# Leaving Old code below - it is not organized

# PORPOW Calibration

In [4]:
porpow_location = r"Validation\PoRPoW\Model"
porpow_path = os.path.join(run_path,porpow_location)

In [5]:
# matrix_list = {}
import_mat = "mf50"
first = True
for root,dirs,files in os.walk(porpow_path):
    for matrix_file in files:
        name = os.path.splitext(matrix_file)[0]
        matrix_number = int(import_mat[2:])
        import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name)
        porpow_mat = eb.matrix(import_mat).get_numpy_data(scenario)
        indices = eb.matrix(import_mat).get_data(1).indices
        if first:
            df_full = pd.DataFrame(porpow_mat,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "TAZ_O",
                                                                                               "level_1": "TAZ_D",
                                                                                              0: name})
            first = False
        else:
            df_curr = pd.DataFrame(porpow_mat,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "TAZ_O",
                                                                                               "level_1": "TAZ_D",
                                                                                              0: name})
            df_full = df_full.merge(df_curr, on= ["TAZ_O","TAZ_D"])

In [16]:
input_pth = r"C:\Users\BCrane\Documents\407 Model Update"
zone_map = pd.read_csv(os.path.join(input_pth,"407_ZoneMapping.csv"))

In [9]:
zone_map_min = zone_map[["DTAZ","pd_new","pd_old", "pd_tts"]]

KeyError: "None of [Index(['DTAZ', 'pd_new', 'pd_old', 'pd_tts'], dtype='object')] are in the [columns]"

In [8]:
df_full = df_full.merge(zone_map_min,left_on = "TAZ_O", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_o", "pd_old":"pd_old_o", "pd_tts":"pd_tts_o"})

In [9]:
df_full = df_full.merge(zone_map_min,left_on = "TAZ_D", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_d", "pd_old":"pd_old_d", "pd_tts":"pd_tts_d"})

In [10]:
all_pivot = pd.pivot_table(df_full[(df_full.pd_tts_o > 0) & (df_full.pd_tts_d > 0)], index = "pd_tts_o", columns = "pd_tts_d", values = ["All"], aggfunc = sum)#.to_csv(os.path.join(input_pth,"model_porpow_all_20210121.csv"))

In [11]:
all_pivot.stack().to_csv(os.path.join(input_pth,"model_new_kfactors4_porpow_all_tall_reaggregate_20210208.csv"))

# Externals

In [24]:
external_location = r"Validation\External"
external_path = os.path.join(run_path,external_location)

In [25]:
auto_external = pd.read_csv(os.path.join(external_path,"AMExternalAuto.csv"))
transit_external = pd.read_csv(os.path.join(external_path,"AMExternalTransit.csv"))

In [31]:
auto_external = auto_external.set_index("Zones O\\D").stack().reset_index().rename(columns = {"Zones O\\D": "O", "level_1": "D", 0: "Auto Trips"})
transit_external = transit_external.set_index("Zones O\\D").stack().reset_index().rename(columns = {"Zones O\\D": "O", "level_1": "D", 0: "Transit Trips"})
auto_external["D"] = auto_external["D"].map(int)
transit_external["D"] = transit_external["D"].map(int)

In [41]:
external_trips = auto_external.merge(transit_external,on=["O","D"])

In [42]:
external_trips = external_trips.merge(zone_map_min,left_on = "O", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_o", "pd_old":"pd_old_o", "pd_tts":"pd_tts_o"})
external_trips = external_trips.merge(zone_map_min,left_on = "D", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_d", "pd_old":"pd_old_d", "pd_tts":"pd_tts_d"})

In [47]:
external_summary = pd.pivot_table(external_trips[(external_trips.pd_tts_o > 0) & (external_trips.pd_tts_d > 0)], index = "pd_tts_o", columns = "pd_tts_d", values = ["Auto Trips"], aggfunc = sum)

In [48]:
external_summary.stack().to_csv(os.path.join(input_pth,"external_auto_trips_20210202.csv"))

In [63]:
for value in zone_map_min["pd_tts"].unique():
    print(value, zone_map_min[zone_map_min["pd_tts"] == value]["pd_new"].unique())

(100, array([1], dtype=int64))
(101, array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64))
(102, array([13, 14, 15, 16], dtype=int64))
(6, array([202], dtype=int64))
(5, array([201, 202], dtype=int64))
(4, array([20], dtype=int64))
(7, array([21], dtype=int64))
(11, array([222], dtype=int64))
(10, array([222, 221], dtype=int64))
(9, array([ 22, 222], dtype=int64))
(14, array([232], dtype=int64))
(13, array([231, 232], dtype=int64))
(12, array([231,  23], dtype=int64))
(15, array([ 24, 241, 242], dtype=int64))
(16, array([241], dtype=int64))
(17, array([242], dtype=int64))
(3, array([19], dtype=int64))
(2, array([18], dtype=int64))
(1, array([17], dtype=int64))
(104, array([33], dtype=int64))
(103, array([29, 31], dtype=int64))
(105, array([28, 27, 32, 30, 26, 25], dtype=int64))
(106, array([34, 35, 36], dtype=int64))
(107, array([39, 40, 38, 37, 44, 43, 45, 46, 41, 42,  0], dtype=int64))


# Checking Persons File

In [129]:
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

In [137]:
persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))

In [138]:
households_df = pd.read_csv(os.path.join(microsim_path,"households.csv"))

In [132]:
persons_df = persons_df.merge(households_df,on="household_id")

In [133]:
persons_df = persons_df.merge(zone_map_min,left_on = "home_zone", right_on ='GTA06').drop(columns = 'GTA06').rename(columns = {"region":"region_home"})

In [134]:
pot_drivers = persons_df[persons_df["age"] >= 16]
license_summary = pd.pivot_table(pot_drivers, index = "region_home", columns = "license", values = "weight_x",aggfunc = sum).fillna(0)

In [135]:
license_summary.to_csv(os.path.join(input_pth,"PorpowRun3_licensesummary.csv"))

In [139]:
def reduce_drivers(drivers):
    if drivers <= 3:
        return drivers
    else:
        return 4
persons_df["license_num"] = persons_df["license"].map({True: 1, False: 0})
license_hh = pd.pivot_table(persons_df, index = "household_id", values = "license_num",aggfunc = sum).reset_index()
household_lic_df = households_df.merge(license_hh, on= "household_id").rename(columns = {"license_num": "drivers"})
household_lic_df = household_lic_df.merge(zone_map_min,left_on = "home_zone", right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_home"})
household_lic_df["driver_mod"] = household_lic_df["drivers"].map(reduce_drivers)
household_lic_summary = pd.pivot_table(household_lic_df, index = ["driver_mod","region_home"], columns = "vehicles", values = "weight", aggfunc = sum).fillna(0)

In [140]:
household_lic_summary.to_csv(os.path.join(input_pth,"porpowrun3_householdvehiclesummary.csv"))

# Mode Share START HERE

In [35]:
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

input_pth = r"C:\Users\pechen\Documents\407 Model Update"
zone_map = pd.read_csv(os.path.join(input_pth,"407_ZoneMapping.csv"))
zone_map_min = zone_map

# def mapTripPurpose(purpose):
#     purpose_list = purpose.split("-")
#     purpose_o = purpose_list[0]
#     purpose_d = purpose_list[1]
#     work = ["PrimaryWork","SecondaryWork"]
#     market = ["Market","JointMarket"]
#     other = ["IndividualOther","JointOther"]
#     if (purpose_o == "Home" or purpose_d == "Home") and (purpose_o in work or purpose_d in work):
#         return "1 - HBW"
#     elif (purpose_o == "Home" or purpose_d == "Home") and (purpose_o == "School" or purpose_d == "School"):
#         return "2 - HBS"
#     elif (purpose_o == 'WorkBasedBusiness' or purpose_d == 'WorkBasedBusiness'):
#         return "3 - WBB"
#     elif (purpose_d in market):
#         return "4 - Market"
#     elif (purpose_d in other):
#         return "5 - Other"
#     elif (purpose_o in market):
#         return "4 - Market"
#     elif (purpose_o in other):
#         return "5 - Other"
#     else:
#         return "6 - Remaining"

def mapTripPurpose(purpose):
    purpose_list = purpose.split("-")
    purpose_o = purpose_list[0]
    purpose_d = purpose_list[1]
    work = ["PrimaryWork","SecondaryWork"]
    market_avoid_d = ['IndividualOther','JointOther','JointMarket']
    other_avoid_d = ['Market','JointOther','JointMarket']
    market_avoid_o = ['JointOther','JointMarket']
    other_avoid_o = ['JointOther','JointMarket']
    
    if (purpose_o == "Home" or purpose_d == "Home") and (purpose_o in work or purpose_d in work):
        return "1 - HBW"
    elif (purpose_o == "Home" or purpose_d == "Home") and (purpose_o == "School" or purpose_d == "School"):
        return "2 - HBS"
    elif (purpose_o in work and purpose_d == 'WorkBasedBusiness'):
        return "3 - WBB"
    elif (purpose_o == "Market" and purpose_d not in market_avoid_d) or (purpose_d == "Market" and purpose_o not in market_avoid_o):
        return "4 - Market"
    elif (purpose_o == "IndividualOther" and purpose_d not in other_avoid_d) or (purpose_d == "IndividualOther" and purpose_o not in other_avoid_o):
        return "5 - Other"
    else:
        return "6 - Remaining"

def specificPurpose(purpose):
    purpose_list = purpose.split("-")
    purpose_o = purpose_list[0]
    purpose_d = purpose_list[1]
    work = ["PrimaryWork","SecondaryWork"]
    market_avoid_d = ['IndividualOther','JointOther','JointMarket']
    other_avoid_d = ['Market','JointOther','JointMarket']
    market_avoid_o = ['JointOther','JointMarket']
    other_avoid_o = ['JointOther','JointMarket']
    
    if (purpose_o in work and purpose_d == 'WorkBasedBusiness'):
        return "1 - WBB"
    elif (purpose_o == "Market" and purpose_d not in market_avoid_d) or (purpose_d == "Market" and purpose_o not in market_avoid_o):
        return "2 - Market"
    elif (purpose_o == "IndividualOther" and purpose_d not in other_avoid_d) or (purpose_d == "IndividualOther" and purpose_o not in other_avoid_o):
        return "3 - Other"
    else:
        return "4 - Remaining"

In [36]:
persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
trips_407 = trips_df
trips_407  = trips_407.merge(persons_df,on=["household_id","person_id"])
trips_407 = trips_407[trips_407.age >=11]
trips_407["combined_act"] = trips_407.o_act + "-" + trips_407.d_act
trips_407 = trips_407.merge(zone_map_min,left_on = "o_zone", right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_o"})
trips_407 = trips_407.merge(zone_map_min,left_on = "d_zone", right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_d"})
trips_407["Purpose"] = trips_407["combined_act"].map(mapTripPurpose)
trip_summary = pd.pivot_table(trips_407,values = "weight_x", index = ["region_o","region_d"], columns = "Purpose", aggfunc = sum, margins = True).fillna(0)

In [37]:
trip_summary.to_csv(os.path.join(input_pth,"dailytripsummary_locationchoicewarm35.csv"))

In [ ]:
trips_407["Purp2"] = trips_407["combined_act"].map(specificPurpose)
trip_summary2 = pd.pivot_table(trips_407,values = "weight_x", index = ["region_o","region_d"], columns = "Purp2", aggfunc = sum).fillna(0)
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
trips_407_time = tripmode_df.merge(trips_407,how = "right", on = ['household_id', 'person_id', 'trip_id'])
trips_407_time["weight"] = trips_407_time["weight"]/10
trips_407_time["weight_z"] = trips_407_time["weight"]*trips_407_time["weight_x"]
trips_407_am = trips_407_time[(trips_407_time.o_depart >= 360) & (trips_407_time.o_depart < 540)]
trips_407_pm = trips_407_time[(trips_407_time.o_depart >= 900) & (trips_407_time.o_depart < 1140)]
trip_summary_am = pd.pivot_table(trips_407_am,values = "weight_z", index = ["region_o","region_d"], columns = "Purp2", aggfunc = sum).fillna(0).reset_index()
trip_summary_pm = pd.pivot_table(trips_407_pm,values = "weight_z", index = ["region_o","region_d"], columns = "Purp2", aggfunc = sum).fillna(0).reset_index()
trip_summary_am = trip_summary_am.rename(columns= {'1 - WBB': 'AM - 1 - WBB', '2 - Market': 'AM - 2 - Market', 
                                 '3 - Other': 'AM - 3 - Other', '4 - Remaining': 'AM - 4 - Remaining'})
trip_summary_pm = trip_summary_pm.rename(columns= {'1 - WBB': 'PM - 1 - WBB', '2 - Market': 'PM - 2 - Market', 
                                 '3 - Other': 'PM - 3 - Other', '4 - Remaining': 'PM - 4 - Remaining'})
trip_summary2 = trip_summary2.merge(trip_summary_am, on=['region_o', 'region_d'], how = "left").merge(trip_summary_pm, on=['region_o', 'region_d'], how = "left").fillna(0)

In [ ]:
trips_407_md = trips_407_time[(trips_407_time.o_depart >= 540) & (trips_407_time.o_depart < 900)]
trips_407_ev = trips_407_time[(trips_407_time.o_depart >= 1140) & (trips_407_time.o_depart < 1440)]
trip_summary_md = pd.pivot_table(trips_407_md,values = "weight_z", index = ["region_o","region_d"], columns = "Purp2", aggfunc = sum).fillna(0).reset_index()
trip_summary_ev = pd.pivot_table(trips_407_ev,values = "weight_z", index = ["region_o","region_d"], columns = "Purp2", aggfunc = sum).fillna(0).reset_index()
trip_summary_md = trip_summary_md.rename(columns= {'1 - WBB': 'MD - 1 - WBB', '2 - Market': 'MD - 2 - Market', 
                                 '3 - Other': 'MD - 3 - Other', '4 - Remaining': 'MD - 4 - Remaining'})
trip_summary_ev = trip_summary_ev.rename(columns= {'1 - WBB': 'EV - 1 - WBB', '2 - Market': 'EV - 2 - Market', 
                                 '3 - Other': 'EV - 3 - Other', '4 - Remaining': 'EV - 4 - Remaining'})
trip_summary3 = trip_summary2.merge(trip_summary_md, on=['region_o', 'region_d'], how = "left").merge(trip_summary_ev, on=['region_o', 'region_d'], how = "left").fillna(0)

In [28]:
trip_summary2.to_csv(os.path.join(input_pth,"daily_ampm_locchoicesummary_locationchoicerun2_lcfix.csv"))

In [ ]:
trip_summary3.to_csv(os.path.join(input_pth,"daily_ampmmdev_locchoicesummary_locationchoicerun2_updated.csv"))

In [8]:
am_modeshare = pd.pivot_table(trips_407_am,index = ["region_o","region_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)#.to_csv(os.path.join(input_pth,"am_modeshare_locationchoicerun4.csv"))
pm_modeshare = pd.pivot_table(trips_407_pm,index = ["region_o","region_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)#.to_csv(os.path.join(input_pth,"am_modeshare_locationchoicerun4.csv"))

In [9]:
am_modeshare.to_csv(os.path.join(input_pth,"am_modeshare_LocationChoiceRun2_updated.csv"))
pm_modeshare.to_csv(os.path.join(input_pth,"pm_modeshare_LocationChoiceRun2_updated.csv"))

In [18]:
pd.pivot_table(trips_407_am,index = ["o_act"],columns = ["d_act"], values = "weight_z", aggfunc = sum,margins = True).fillna(0)

d_act,Home,IndividualOther,JointMarket,JointOther,Market,PrimaryWork,ReturnFromWork,School,SecondaryWork,WorkBasedBusiness,All
o_act,,,,,,,,,,,
Home,0.0,220788.4,2821.6,9327.4,39355.1,1.843663e+06,0.0,648188.099995,0.0,30651.6,2.794795e+06
IndividualOther,41326.0,4425.8,67.2,196.2,670.1,2.578370e+04,4.3,6034.600000,0.0,301.4,7.880930e+04
JointMarket,109.2,31.9,21.4,9.3,25.5,3.200000e+00,0.0,1.000000,0.0,1.0,2.025000e+02
JointOther,1065.2,81.2,8.9,68.2,27.2,6.960000e+01,0.0,4.300000,0.0,0.0,1.324600e+03
Market,4316.0,727.3,18.3,38.9,347.1,1.883500e+03,2.0,148.100000,0.0,24.3,7.505500e+03
PrimaryWork,7452.0,310.3,1.0,0.5,80.3,1.967000e+02,2996.3,1734.300000,1.0,46171.3,5.894370e+04
School,1662.0,10.2,0.0,0.0,2.6,7.852000e+02,6.2,548.800000,0.0,17.2,3.032200e+03
WorkBasedBusiness,2252.0,56.6,0.0,0.0,13.1,1.500480e+04,0.0,69.100000,0.0,851.2,1.824680e+04
All,58182.4,226431.7,2938.4,9640.5,40521.0,1.887389e+06,3008.8,656728.299995,1.0,78018.0,2.962859e+06


In [10]:
trip_summary_am2 = pd.pivot_table(trips_407_am,values = "weight_z", index = ["region_o","region_d"], columns = "Purpose", aggfunc = sum).fillna(0).reset_index()
trip_summary_pm2 = pd.pivot_table(trips_407_pm,values = "weight_z", index = ["region_o","region_d"], columns = "Purpose", aggfunc = sum).fillna(0).reset_index()

In [11]:
trip_summary_am2.to_csv(os.path.join(input_pth,"am_trips_LocationChoiceRun2_updated.csv"))
trip_summary_pm2.to_csv(os.path.join(input_pth,"pm_trips_LocationChoiceRun2_updated.csv"))

In [21]:
trip_summary_md2 = pd.pivot_table(trips_407_md,values = "weight_z", index = ["region_o","region_d"], columns = "Purpose", aggfunc = sum).fillna(0).reset_index()
trip_summary_ev2 = pd.pivot_table(trips_407_ev,values = "weight_z", index = ["region_o","region_d"], columns = "Purpose", aggfunc = sum).fillna(0).reset_index()

In [22]:
trip_summary_md2.to_csv(os.path.join(input_pth,"md_trips_LocationChoiceRun2.csv"))
trip_summary_ev2.to_csv(os.path.join(input_pth,"ev_trips_LocationChoiceRun2.csv"))

In [12]:
persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
trips_gta = trips_df[(trips_df.o_zone < 6000) & (trips_df.d_zone < 6000)]

In [13]:
trips_gta = trips_gta.merge(persons_df,on=["household_id","person_id"])
trips_gta = trips_gta[trips_gta.age >=11]

In [14]:
trips_gta_time = tripmode_df.merge(trips_gta,how = "right", on = ['household_id', 'person_id', 'trip_id'])

In [15]:
trips_gta_time["weight"] = trips_gta_time["weight"]/10
trips_gta_time["weight_z"] = trips_gta_time["weight"]*trips_gta_time["weight_x"]

In [16]:
pd.pivot_table(trips_gta_time,columns = "mode", values = "weight_z", aggfunc = sum).to_csv(os.path.join(input_pth,"GTA_Wide_TripsbyMode_LocationChoiceRun2_updatedagain.csv"))

In [76]:
trips_407[trips_407["Purpose"] =="6 - Remaining"]["combined_act"].unique()

array(['SecondaryWork-PrimaryWork', 'PrimaryWork-SecondaryWork',
       'PrimaryWork-PrimaryWork', 'School-PrimaryWork',
       'PrimaryWork-ReturnFromWork', 'School-SecondaryWork',
       'PrimaryWork-School', 'School-School', 'SecondaryWork-School',
       'School-ReturnFromWork'], dtype=object)

In [100]:
tor_york_trips = trips_407[(trips_407["region_o"] == 1) & (trips_407["region_d"] == 3) & (trips_407["Purpose"] != "1 - HBW") & (trips_407["Purpose"] != "2 - HBS")]

In [106]:
tor_york_trips["weight_x"].sum()

235994

In [102]:
tor_york_trips = tor_york_trips.merge(households_df,on="household_id")

In [104]:
tor_york_trips[tor_york_trips["work_zone"] == 0]["weight_x"].sum()

105189

In [110]:
pd.pivot_table(tor_york_trips[tor_york_trips["work_zone"] == 0], index = "employment_status", columns = "student_status",values= "weight_x", aggfunc = sum).fillna(0)

student_status,F,O,P
employment_status,,,
O,16527,87311,1351


In [108]:
pd.pivot_table(tor_york_trips[tor_york_trips["work_zone"] == 0], index = "region_home", values= "weight_x", aggfunc = sum).fillna(0)

,weight_x
region_home,
1,50739
2,226
3,53557
4,648
5,18
6,1


In [105]:
tor_york_trips = tor_york_trips.merge(zone_map_min,left_on = 'home_zone', right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_home"})

In [98]:
tor_york_trips = tor_york_trips.merge(zone_map_min,left_on = 'home_zone', right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_home"})
tor_york_trips = tor_york_trips.merge(zone_map_min,left_on = 'work_zone', right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_work"})

In [90]:
pd.pivot_table(tor_york_trips, index = "region_home", columns = "region_work", values= "weight_x", aggfunc = sum).fillna(0)

region_work,1,2,3,4,5,6,7
region_home,,,,,,,
1,21719.0,159.0,9521.0,1352.0,61.0,8.0,87.0
2,1167.0,134.0,356.0,16.0,1.0,0.0,7.0
3,44055.0,745.0,37854.0,3926.0,213.0,20.0,341.0
4,1176.0,3.0,406.0,271.0,7.0,0.0,8.0
5,145.0,0.0,29.0,7.0,13.0,0.0,0.0
6,27.0,0.0,12.0,0.0,1.0,0.0,0.0


In [24]:
pd1_trips = durham_trips_am[(durham_trips_am["pd_tts_d"] == 100) & ((durham_trips_am["pd_tts_o"] == 4) | (durham_trips_am["pd_tts_o"] == 7)| (durham_trips_am["pd_tts_o"] == 9)| (durham_trips_am["pd_tts_o"] == 12)| (durham_trips_am["pd_tts_o"] == 17))]

In [17]:
pd1_trips = durham_trips_pm[(durham_trips_pm["pd_tts_o"] == 100) & ((durham_trips_pm["pd_tts_d"] == 4) | (durham_trips_pm["pd_tts_d"] == 7)| (durham_trips_pm["pd_tts_d"] == 9)| (durham_trips_pm["pd_tts_d"] == 12)| (durham_trips_pm["pd_tts_d"] == 17))]

In [25]:
pd1_trips[pd1_trips["mode"] == "DAT"]["weight"].sum(),pd1_trips[pd1_trips["mode"] == "PAT"]["weight"].sum(),pd1_trips[pd1_trips["mode"] == "PET"]["weight"].sum()

(2978.1000000000004, 8429.099999999999, 697.6)

In [18]:
pd1_trips["weight"].sum()

16612.6

In [11]:
durham_trips_pm.columns

Index(['household_id', 'person_id', 'trip_id', 'mode', 'o_depart', 'd_arrive',
       'weight', 'o_act', 'o_zone', 'd_act', 'd_zone', 'weight_x', 'age',
       'sex', 'license', 'transit_pass', 'employment_status', 'occupation',
       'free_parking', 'student_status', 'work_zone', 'school_zone',
       'weight_y', 'combined_act', 'pd_new_o', 'pd_old_o', 'pd_tts_o',
       'pd_new_d', 'pd_old_d', 'pd_tts_d', 'Purpose', 'Purp2'],
      dtype='object')

In [153]:
int_pickering = durham_trips_pm[(durham_trips_pm["pd_tts_o"] == 9) & (durham_trips_pm["pd_tts_d"] == 9)]

In [154]:
wat_int_pickering = int_pickering[int_pickering["mode"] == "WAT"]

In [155]:
wat_int_pickering

,household_id,person_id,trip_id,mode,o_depart,d_arrive,weight,o_act,o_zone,d_act,...,weight_y,combined_act,pd_new_o,pd_old_o,pd_tts_o,pd_new_d,pd_old_d,pd_tts_d,Purpose,Purp2
4324,420756,3,2,WAT,1110.3020,1140.000,0.6,PrimaryWork,1302,IndividualOther,...,1,PrimaryWork-IndividualOther,22,22,9,22,22,9,5 - Other,3 - Other
10241,718952,2,2,WAT,929.8954,945.000,0.1,PrimaryWork,1383,IndividualOther,...,1,PrimaryWork-IndividualOther,22,22,9,22,22,9,5 - Other,3 - Other
11626,811607,1,2,WAT,1092.7140,1143.586,0.4,PrimaryWork,1402,Market,...,1,PrimaryWork-Market,22,22,9,22,22,9,4 - Market,2 - Market
12986,848847,1,2,WAT,975.2733,1035.000,0.8,PrimaryWork,1347,Market,...,1,PrimaryWork-Market,22,22,9,22,22,9,4 - Market,2 - Market
38036,1009892,1,2,WAT,967.4708,1024.630,0.4,IndividualOther,1344,IndividualOther,...,1,IndividualOther-IndividualOther,22,22,9,22,22,9,5 - Other,3 - Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986148,1325601,1,2,WAT,989.1961,1017.222,0.1,PrimaryWork,1384,Market,...,1,PrimaryWork-Market,22,22,9,22,22,9,4 - Market,2 - Market
1986150,1325601,1,3,WAT,1110.8360,1140.000,0.1,Market,1383,SecondaryWork,...,1,Market-SecondaryWork,22,22,9,22,22,9,4 - Market,2 - Market
1999404,1327518,1,2,WAT,956.7364,1027.180,0.2,PrimaryWork,1402,Market,...,1,PrimaryWork-Market,22,22,9,22,22,9,4 - Market,2 - Market
2061454,1336124,4,2,WAT,932.3376,960.000,0.5,School,1371,IndividualOther,...,1,School-IndividualOther,22,22,9,22,22,9,5 - Other,3 - Other


In [147]:
households_df = pd.read_csv(os.path.join(microsim_path,"households.csv"))

In [156]:
wat_int_pickering=wat_int_pickering.rename(columns = {"weight":"weight_trip"})

In [157]:
wat_int_pickering = wat_int_pickering.merge(households_df, on = "household_id", how = "left")

In [159]:
pd.pivot_table(wat_int_pickering, values = "weight_trip", aggfunc = sum,index = "student_status").fillna(0)#.to_csv(os.path.join(input_pth, "pickeringtrip_test.csv"))

,weight_trip
student_status,
F,1217.1
O,762.4
P,26.9


In [124]:
wat_int_pickering.columns

Index(['household_id', 'person_id', 'trip_id', 'mode', 'o_depart', 'd_arrive',
       'weight_x', 'o_act', 'o_zone', 'd_act', 'd_zone', 'weight_x', 'age',
       'sex', 'license', 'transit_pass', 'employment_status', 'occupation',
       'free_parking', 'student_status', 'work_zone', 'school_zone',
       'weight_y', 'combined_act', 'pd_new_o', 'pd_old_o', 'pd_tts_o',
       'pd_new_d', 'pd_old_d', 'pd_tts_d', 'Purpose', 'Purp2', 'home_zone',
       'weight_y', 'persons', 'dwelling_type', 'vehicles', 'income_class'],
      dtype='object')

In [36]:
durham_trips_am["mode"].unique()

array(['Auto', 'WAT', 'DAT', 'Carpool', 'Walk', 'PAT', 'VFH', 'Passenger',
       'PET', 'Schoolbus', 'RideShare', 'Bicycle'], dtype=object)

In [15]:
durham_trips_am.columns

Index([u'household_id', u'person_id', u'trip_id', u'mode', u'o_depart',
       u'd_arrive', u'weight', u'o_act', u'o_zone', u'd_act', u'd_zone',
       u'weight_x', u'age', u'sex', u'license', u'transit_pass',
       u'employment_status', u'occupation', u'free_parking', u'student_status',
       u'work_zone', u'school_zone', u'weight_y', u'combined_act', u'pd_new_o',
       u'pd_old_o', u'pd_tts_o', u'pd_new_d', u'pd_old_d', u'pd_tts_d',
       u'Purpose', u'Purp2'],
      dtype='object')

In [24]:
durham_trips_time[(durham_trips_time.household_id == 468) & (durham_trips_time.person_id == 2)][[u'household_id',u'person_id',u'o_depart',"o_zone","d_zone","mode","weight"]]

,household_id,person_id,o_depart,o_zone,d_zone,mode,weight
0,468,2,440.2754,1,1512,Auto,0.6
1,468,2,480.0000,1,1512,WAT,0.2
2,468,2,374.0309,1,1512,DAT,0.1
3,468,2,440.2754,1,1512,Carpool,0.1
4,468,2,1020.0000,1512,1,Auto,0.6
5,468,2,1020.0000,1512,1,Carpool,0.1
6,468,2,1020.0000,1512,1,DAT,0.1
7,468,2,1020.0000,1512,1,WAT,0.2


In [ ]:
durham_trips_time[durham_trips_am["mode"] == "DAT"]

In [19]:
durham_trips_am[durham_trips_am["mode"] == "DAT"][[u'household_id',u'person_id',u'o_depart',"o_zone","d_zone","mode"]]

,household_id,person_id,o_depart,o_zone,d_zone,mode
2,468,2,374.0309,1,1512,DAT
9,621,1,474.1747,514,1133,DAT
27,1017,1,444.4991,5,1134,DAT
31,1017,2,365.7167,5,1134,DAT
47,3597,2,469.6289,16,1129,DAT
92,6172,1,381.0441,17,1217,DAT
96,6770,1,373.2345,17,1116,DAT
99,6770,1,420.2014,1116,530,DAT
102,6914,1,529.2610,17,1180,DAT
126,8257,1,386.2789,17,1134,DAT


In [26]:
durham_trips_time.columns

Index([u'household_id', u'person_id', u'trip_id', u'mode', u'o_depart',
       u'd_arrive', u'weight', u'o_act', u'o_zone', u'd_act', u'd_zone',
       u'weight_x', u'age', u'sex', u'license', u'transit_pass',
       u'employment_status', u'occupation', u'free_parking', u'student_status',
       u'work_zone', u'school_zone', u'weight_y', u'combined_act', u'pd_new_o',
       u'pd_old_o', u'pd_tts_o', u'pd_new_d', u'pd_old_d', u'pd_tts_d',
       u'Purpose', u'Purp2'],
      dtype='object')

In [34]:
durham_trips_am[(durham_trips_am["mode"] == "DAT")]["weight"].sum(),durham_trips_pm[(durham_trips_pm["mode"] == "DAT")]["weight"].sum()

(19376.300000000003, 27679.900000000005)

In [36]:
pd.pivot_table(durham_trips_pm[(durham_trips_pm["mode"] == "DAT")], index = "pd_tts_o", columns = "pd_tts_d", values = "weight", aggfunc = sum).fillna(0).to_csv(os.path.join(input_pth,"Durham_DAT_PM.csv"))

In [9]:
durham_trips_pm["mode"].unique()

array(['Auto', 'Carpool', 'DAT', 'WAT', 'PAT', 'VFH', 'Passenger', 'PET',
       'Walk', 'RideShare', 'Schoolbus', 'Bicycle'], dtype=object)

In [74]:
durham_trips[durham_trips.Purpose == "2 - HBS"].combined_act.unique()

array(['Home-School', 'School-Home'], dtype=object)

In [15]:
durham_trips.o_act.unique()

array(['Home', 'PrimaryWork', 'WorkBasedBusiness', 'IndividualOther',
       'Market', 'School', 'JointOther', 'SecondaryWork', 'JointMarket'],
      dtype=object)

In [130]:
durham_trips[durham_trips.d_act == 'Market'].o_act.unique()

array(['Home', 'PrimaryWork', 'Market', 'IndividualOther',
       'WorkBasedBusiness', 'JointMarket', 'JointOther', 'School',
       'SecondaryWork'], dtype=object)

In [128]:
len(durham_trips[durham_trips.d_act == 'IndividualOther'])

211731

In [114]:
durham_trips.head()

,household_id,person_id,trip_id,o_act,o_zone,d_act,d_zone,weight_x,age,sex,...,school_zone,weight_y,combined_act,pd_new_o,pd_old_o,pd_tts_o,pd_new_d,pd_old_d,pd_tts_d,Purpose
0,468,2,1,Home,1,PrimaryWork,1515,1,24,F,...,0,1,Home-PrimaryWork,1,1,100,23,23,12,1 - HBW
1,924577,1,2,PrimaryWork,1515,WorkBasedBusiness,1515,1,38,F,...,0,1,PrimaryWork-WorkBasedBusiness,23,23,12,23,23,12,3 - WBB
2,924577,1,3,WorkBasedBusiness,1515,PrimaryWork,1515,1,38,F,...,0,1,WorkBasedBusiness-PrimaryWork,23,23,12,23,23,12,3 - WBB
3,1177781,7,2,School,1515,School,1515,1,14,F,...,1515,1,School-School,23,23,12,23,23,12,6 - Remaining
4,1189434,3,2,PrimaryWork,1515,WorkBasedBusiness,1515,1,26,M,...,0,1,PrimaryWork-WorkBasedBusiness,23,23,12,23,23,12,3 - WBB


In [ ]:
durham_trips[]

In [20]:
durham_trips[durham_trips.d_act == "ReturnFromWork"]["combined_act"].unique()

array(['PrimaryWork-ReturnFromWork', 'IndividualOther-ReturnFromWork',
       'Market-ReturnFromWork', 'School-ReturnFromWork'], dtype=object)

In [7]:
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))

In [8]:
tripmode_df.head()

,household_id,person_id,trip_id,mode,o_depart,d_arrive,weight
0,6,1,1,Auto,512.9304,540.0000,6
1,1,1,1,Auto,729.0034,735.0000,8
2,1,1,1,WAT,707.6055,735.0000,2
3,1,1,2,Auto,990.0000,998.3496,8
4,1,1,2,WAT,990.0000,1024.1150,1


# XML Edits HERE

In [26]:
import xml.etree.ElementTree as ET

In [27]:
model_syst_pth = os.path.join(input_pth,"Python Model Systems")

In [105]:
constant_df = pd.read_csv(os.path.join(model_syst_pth,"durham_msconstants_spatialrun46.csv"))

In [106]:
constant_df["OD"] = constant_df["O_PD"].map(str) + ":" + constant_df["D_PD"].map(str)

In [107]:
constant_df.head()

,Mode,TP,O_PD,D_PD,Constant,OD
0,Bicycle,AM,21,21,-8.0,21:21
1,Carpool,AM,21,21,-12.0,21:21
2,Schoolbus,AM,21,21,-10.0,21:21
3,VFH,AM,21,21,-12.0,21:21
4,Walk,AM,21,21,-8.0,21:21


In [108]:
xml_fle = os.path.join(model_syst_pth,"Durham_Calibration_SpatialRun35_FixedPM_20210318.xml")
tree = ET.parse(xml_fle)
root = tree.getroot()
model = root[1]
otherModes = model[6]

In [109]:
for index, row in constant_df.iterrows():
    mode = row.Mode
    tp = row.TP
    od = row.OD
    const = row.Constant
    match = False
    for child in otherModes:
        curr_mode = child.attrib["Name"]
        if curr_mode == mode:
            mode_tree = child
            match = True
    if not match:
        print("No match found for {}".format(mode))
    else:
        match = False
        for child in mode_tree:
            if "Name" in child.attrib.keys():
                if child.attrib["Name"] == 'Time Period Constants':
                    tp_tree = child
                    match = True
        if not match:
            print("No time period constants for {}".format(mode))
        else:
            match = False
            for child in tp_tree:
                if "Name" in child.attrib.keys():
                    if child.attrib["Name"] == tp:
                        this_tp_tree = child[1]
                        match = True
            if not match:
                print("Time period {0} does not exist for mode {1}".format(tp,mode))
            else:
                match = False
                for child in this_tp_tree:
                    if "Name" in child.attrib.keys():
                        if child.attrib["Name"] == od:
                            match = True
                            const_tree = child
                if not match:
                    print("OD {0} not present for tp {1} mode {2}".format(od,tp,mode))
                else:
                    match = False
                    params = const_tree[0]
                    for child in params:
                        if child.attrib["Name"] == "Constant":
                            param = child
                            match = True
                    if not match:
                        print("Constant attribute not present for od {0} tp {1} mode {2}".format(od,tp,mode))
                    else:
                        param.attrib["Value"] = str(const)
                

In [110]:
tree.write(os.path.join(model_syst_pth,"Durham_Calibration_SpatialRun46_20210323.xml"))

In [100]:
gen_const_df = pd.read_csv(os.path.join(model_syst_pth,"durham_msconstants_genrun9.csv"))

In [102]:
gen_const_df.head()

,Mode,Const_Name,Const
0,WAT,GeneralConstant,0.979962
1,WAT,ManufacturingConstant,-1.997008
2,WAT,NonWorkerStudentConstant,-1.410627
3,WAT,ProfessionalConstant,-1.769111
4,WAT,SalesConstant,-1.823318


In [103]:
xml_fle = os.path.join(model_syst_pth,"Durham_Calibration_SpatialRun9_20210308.xml")
tree = ET.parse(xml_fle)
root = tree.getroot()
model = root[1]
otherModes = model[6]

In [105]:
for index, row in gen_const_df.iterrows():
    mode = row.Mode
    const_name = row.Const_Name
    const = row.Const
    for child in otherModes:
        curr_mode = child.attrib["Name"]
        if curr_mode == mode:
            params = child[0]
    for child in params:
        if child.attrib["Name"] == const_name:
            child.attrib["Value"] = str(const)

In [106]:
tree.write(os.path.join(model_syst_pth,"Durham_Calibration_GeneralRun9_20210309.xml"))

In [153]:
params = const_tree[0]

In [154]:
for child in params:
    print(child.tag," - " ,child.attrib)

Param  -  {'Name': 'Origins', 'TIndex': '0', 'Value': '20'}
Param  -  {'Name': 'Destinations', 'TIndex': '0', 'Value': '2-16,25-46'}
Param  -  {'Name': 'Constant', 'TIndex': '18', 'Value': '-1'}


In [137]:
constant_df.columns

Index(['Mode', 'TP', 'O_PD', 'D_PD', 'Constant', 'OD'], dtype='object')

In [116]:
carpool = otherModes[0]

In [117]:
for child in carpool:
    print(child.tag," - " ,child.attrib)

Parameters  -  {}
Collection  -  {'ParentTIndex': '115', 'ParentFieldName': 'TimePeriodConstants', 'Name': 'Time Period Constants'}


In [118]:
carpool[1]

<Element 'Collection' at 0x000001D2E10B39F8>

In [141]:
for child in carpool[1]:
    print(child.tag," - " ,child.attrib)

Module  -  {'Name': 'AM', 'TIndex': '114', 'ParentTIndex': '114', 'ParentFieldName': 'TimePeriodConstants'}


In [143]:
for child in carpool[1][0][1]:
     print(child.tag," - " ,child.attrib)

Module  -  {'Name': '20:21,22,23,242', 'TIndex': '112', 'ParentTIndex': '112', 'ParentFieldName': 'SpatialConstants'}
Module  -  {'Name': '21:20,22,23,242', 'TIndex': '112', 'ParentTIndex': '112', 'ParentFieldName': 'SpatialConstants'}
Module  -  {'Name': '22:20,21,23,242', 'TIndex': '112', 'ParentTIndex': '112', 'ParentFieldName': 'SpatialConstants'}
Module  -  {'Name': '23:20,21,22,242', 'TIndex': '112', 'ParentTIndex': '112', 'ParentFieldName': 'SpatialConstants'}
Module  -  {'Name': '20:2-16,25-46', 'TIndex': '112', 'ParentTIndex': '112', 'ParentFieldName': 'SpatialConstants'}
Module  -  {'Name': '21:2-16,25-46', 'TIndex': '112', 'ParentTIndex': '112', 'ParentFieldName': 'SpatialConstants'}


In [88]:
walk = otherModes[2]

In [89]:
for child in walk:
    print(child.tag," - " ,child.attrib)

Parameters  -  {}
Module  -  {'Name': 'Custom Distances', 'Description': "A custom set of distances(m) if the paths differ from the zone system's distance matrix", 'TIndex': '-1', 'ParentTIndex': '227', 'ParentFieldName': 'CustomDistances'}
Collection  -  {'ParentTIndex': '115', 'ParentFieldName': 'TimePeriodConstants', 'Name': 'Time Period Constants'}
Module  -  {'Name': 'Zonal Destination Utility', 'Description': 'An optional utility modification when traveling to the given zone.', 'TIndex': '-1', 'ParentTIndex': '226', 'ParentFieldName': 'ZonalDestinationUtility'}


In [103]:
tp_consts = walk[3]

In [105]:
for child in tp_consts:
    print(child.tag," - " ,child.attrib)

Parameters  -  {}
